# Setup

IMPORTANT NOTES:
- Your domino instance MUST be running on AWS.
- Your domino instance MUST be in the same AWS region as your model deployment for this initial preview capability.

## 1. Deployment Setup

### 1a. Create or upgrade deployment
Use the latest HEAD or a 6.0+ catalog with an AWS deployment flavor.

### 1b. Enable feature flag
Enable the `ShortLived.EnableModelDeploymentsApi` feature flag on your domino deployment.

## 2. AWS Account/Role Setup

### 2a. Create AWS resources (including user & roles) to use for deployment to sagemaker
NOTE #1: skip if using `dmo-team-sandbox` and use IAM user `sagemaker-tests` and IAM role `sagemakerFullAccess`
NOTE #2: you must have full admin access to the AWS accounts to perform these steps
1. Run the following script to create an IAM role/user and setup requierd policies in the *target* account:
    - Script: [target_account_setup.sh](https://github.com/cerebrotech/pham-juno-operator-service/blob/main/dev/setup-scripts/target_account_setup.sh)
    - If you're using `dmo-team-sandbox` as the domino and target account, you can skip the IAM user/role creation step and use the existing `dmo-team-sandbox` IAM user `sagemaker-tests` and IAM role `sagemakerFullAccess`.  You WILL need to run step #2 below to assign the correct permissions for the IAM role to decrypt the domino blob storage s3 bucket for your deployment.
2. Run the following script to create an IAM role/user and setup requierd policies in the *domino* account (where Domino itself is running:
    - Script: [domino_account_setup.sh](https://github.com/cerebrotech/pham-juno-operator-service/blob/main/dev/setup-scripts/domino_account_setup.sh)


## 3. Project Setup

### 3a. Create project from this git repo
1. Login to the domino deployment
2. Add git credentials for github
3. Create a new git-based project using the repo https://github.com/cerebrotech/sagemaker-deployment-demo

### 3b. Setup Env Vars for AWS creds
For the project you just created, add project environment variables for AWS creds for the user in the target AWS account.  (If using `sagemaker-tests` in `dmo-team-sandbox`, get these values from [this shared 1Password credential](https://start.1password.com/open/i?a=5TQ765YJWFGZRMYI7SQGSLI4EI&v=ofq5ozbsgrpewd3g5jzw4ygtkq&i=vsh3zr6l4gxcyfjcpedf6wzwua&h=dominodatalab.1password.com).)
1. `AWS_ACCESS_KEY_ID`: populate from `access key` field
2. `AWS_SECRET_ACCESS_KEY`: populate from `secret access key` field

If you are already running a workspace, stop and restart the workspace after adding these variables.

### 3c. Create custom environment
Create a custom environment with the packages needed for the sentiment analysis model derived from the DME and update your project to use this environment:
1. Environment Base: "Domino Core Environment"
2. Dockerfile instructions:
    ```dockerfile
    USER root
    RUN pip install "mlflow==2.13.2" "transformers<=4.39.3" datasets ipywidgets torch torchvision boto3 accelerate --upgrade
    USER ubuntu
    ```
3. Pluggable workspace tools (or copy all from DCE):
    ```
    jupyter:
      title: "Jupyter (Python, R, Julia)"
      iconUrl: "/assets/images/workspace-logos/Jupyter.svg"
      start: [ "/opt/domino/workspaces/jupyter/start" ]
      supportedFileExtensions: [ ".ipynb" ]
      httpProxy:
        port: 8888
        rewrite: false
        internalPath: "/{{ownerUsername}}/{{projectName}}/{{sessionPathComponent}}/{{runId}}/{{#if pathToOpen}}tree/{{pathToOpen}}{{/if}}"
        requireSubdomain: false
    ```
4. Confirm that the image builds successfully
5. Change the `sagemaker-deployment-demo` project's default environment to this custom enviroment.
    - If you are already running a workspace, create a new the workspace after changing this environment.